<a href="https://colab.research.google.com/github/bh33oct/python-programming-task/blob/main/BHoomi_code_aplha_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

def choose_word():
    words = ['python', 'hangman', 'programming', 'developer', 'challenge', 'computer', 'science']
    return random.choice(words)

def display_hangman(tries):
    stages = [
        """
           ------
           |    |
           |    O
           |   /|\\
           |   / \\
           |
        """,
        """
           ------
           |    |
           |    O
           |   /|\\
           |   /
           |
        """,
        """
           ------
           |    |
           |    O
           |   /|
           |
           |
        """,
        """
           ------
           |    |
           |    O
           |    |
           |
           |
        """,
        """
           ------
           |    |
           |    O
           |
           |
           |
        """,
        """
           ------
           |    |
           |
           |
           |
           |
        """,
        """
           ------
           |
           |
           |
           |
           |
        """,
    ]
    return stages[tries]

def play_hangman():
    word = choose_word()
    word_completion = "_" * len(word)
    guessed = False
    guessed_letters = []
    guessed_words = []
    tries = 6

    print("Let's play Hangman!")
    print(display_hangman(tries))
    print(word_completion)
    print("\n")

    while not guessed and tries > 0:
        guess = input("Please guess a letter or word: ").lower()

        if len(guess) == 1 and guess.isalpha():
            if guess in guessed_letters:
                print("You already guessed that letter. Try again.")
            elif guess not in word:
                print(f"{guess} is not in the word.")
                tries -= 1
                guessed_letters.append(guess)
            else:
                print(f"Good job! {guess} is in the word.")
                guessed_letters.append(guess)
                word_completion = "".join([letter if letter in guessed_letters else "_" for letter in word])

                if "_" not in word_completion:
                    guessed = True
        elif len(guess) == len(word) and guess.isalpha():
            if guess in guessed_words:
                print("You already guessed that word. Try again.")
            elif guess != word:
                print(f"{guess} is not the word.")
                tries -= 1
                guessed_words.append(guess)
            else:
                guessed = True
                word_completion = word
        else:
            print("Invalid input. Please enter a single letter or a full word.")

        print(display_hangman(tries))
        print(word_completion)
        print("\n")

    if guessed:
        print(f"Congratulations! You've guessed the word: {word}")
    else:
        print(f"Sorry, you've run out of tries. The word was: {word}")

if _name_ == "_main_":
    play_hangman()

NameError: name '_name_' is not defined

In [ ]:
import nltk
import random
import re
import string

# Download necessary NLTK resources
nltk.download('punkt')

# Define some simple patterns and responses
responses = {
    r'hi|hello|hey': ["Hello!", "Hi there!", "Hey! How can I help you?"],
    r'how are you?': ["I'm just a computer program, but thanks for asking!", "Doing well, how about you?"],
    r'what is your name?': ["I'm a chatbot created to assist you.", "You can call me Chatbot."],
    r'bye|exit|quit': ["Goodbye! Have a great day!", "See you later!"],
    r'(.*)': ["I'm sorry, I don't understand that.", "Can you rephrase that?"]
}

def chatbot_response(user_input):
    # Normalize the input
    user_input = user_input.lower()

    # Check for matching patterns
    for pattern, response_list in responses.items():
        if re.match(pattern, user_input):
            return random.choice(response_list)

    return "I'm not sure how to respond to that."

def main():
    print("Chatbot: Hello! I'm here to chat with you. Type 'bye' to exit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['bye', 'exit', 'quit']:
            print("Chatbot: Goodbye! Have a great day!")
            break

        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")

if _name_ == "_main_":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: name '_name_' is not defined

In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import alpha_vantage

app = Flask(_name_)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///portfolio.db"
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True)
    password = db.Column(db.String(128))

class Portfolio(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    created_date = db.Column(db.DateTime)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"))

class Stock(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    ticker_symbol = db.Column(db.String(64), unique=True)
    quantity = db.Column(db.Integer)
    purchase_date = db.Column(db.DateTime)
    portfolio_id = db.Column(db.Integer, db.ForeignKey("portfolio.id"))

@app.route("/users", methods=["GET"])
def get_users():
    users = User.query.all()
    return jsonify([user.to_dict() for user in users])

@app.route("/portfolios", methods=["GET"])
def get_portfolios():
    portfolios = Portfolio.query.all()
    return jsonify([portfolio.to_dict() for portfolio in portfolios])

@app.route("/stocks", methods=["GET"])
def get_stocks():
    stocks = Stock.query.all()
    return jsonify([stock.to_dict() for stock in stocks])

@app.route("/stocks", methods=["POST"])
def add_stock():
    ticker_symbol = request.json["ticker_symbol"]
    quantity = request.json["quantity"]
    portfolio_id = request.json["portfolio_id"]
    stock = Stock(ticker_symbol=ticker_symbol, quantity=quantity, portfolio_id=portfolio_id)
    db.session.add(stock)
    db.session.commit()
    return jsonify({"message": "Stock added successfully"})

@app.route("/stocks/<ticker_symbol>", methods=["GET"])
def get_stock(ticker_symbol):
    stock = Stock.query.filter_by(ticker_symbol=ticker_symbol).first()
    if stock:
        return jsonify(stock.to_dict())
    return jsonify({"message": "Stock not found"})

@app.route("/api/stock_data", methods=["GET"])
def get_stock_data():
    ticker_symbol = request.args.get("ticker_symbol")
    api_key = "YOUR_API_KEY_HERE"
    alpha_vantage_api = alpha_vantage.AlphaVantage(api_key)
    data = alpha_vantage_api.get_stock_data(ticker_symbol)
    return jsonify(data)

if _name_ == "_main_":
    app.run(debug=True)